In [34]:
from logging import Logger
from dotenv import load_dotenv
import json
import google.generativeai as genai
import pandas as pd
import os
import numpy as np
import logging
import typing_extensions as typing
import time
from app.utils.stomach import save_json

In [35]:
logger = logging.getLogger(__name__)  # Use a named logger

In [4]:
logging.basicConfig(level=logging.INFO)
RECIPE = "../../app/data/processed/recipe_cleaned.json"
INGREDIENTS = "../../app/data/processed/ingredients_AI.json"
MEAL = "../../app/data/processed/meal_cleaned.json"
MEALTORECIPE = "../../app/data/processed/mealtorecipe.json"
RECIPESTEP = "../../app/data/processed/recipestep.json" 

In [5]:
logger.info("PREPARING DATA")
data: pd.DataFrame = pd.read_json(MEAL)
data = data.replace({np.nan: None, '': None, ' ': None})

INFO:__main__:PREPARING DATA


In [6]:
data["updated_at"] = data["created_at"]

In [7]:
data["status"] = "PUBLISHED"
data["video_url"] = "Not Available"

In [8]:
data = data[["meal_id", "name", "description", "status", "created_at", "updated_at", "source", "video_url"]]
len(data)

220

In [9]:
df_cleaned = data.drop_duplicates(subset=["name", "description"])
df_cleaned = df_cleaned.reset_index().drop(columns="index")
len(df_cleaned)

112

In [30]:
import pandas as pd

# Load datasets
meals_df = df_cleaned
recipes_df = pd.read_json(RECIPE)
meal_to_recipe_df = pd.read_json(MEALTORECIPE)
ingredients_df = pd.read_json(INGREDIENTS)
recipe_step_df = pd.read_json(RECIPESTEP)

recipes_df = recipes_df.drop_duplicates()
print(len(recipes_df))

meals_df.columns, recipes_df.columns, ingredients_df.columns, meal_to_recipe_df.columns, recipe_step_df.columns

220


(Index(['meal_id', 'name', 'description', 'status', 'created_at', 'updated_at',
        'source', 'video_url'],
       dtype='object'),
 Index(['recipe_id', 'course', 'cuisine', 'servings', 'calories', 'prep time',
        'cook time', 'total time', 'author', 'keyword', 'soaking time',
        'marinating time', 'cooling time', 'resting time', 'wait time'],
       dtype='object'),
 Index(['main_ingredient', 'quantity', 'weight', 'ingredient_id',
        'alternative_ingredient', 'measurement'],
       dtype='object'),
 Index(['meal_id', 'recipe_id'], dtype='object'),
 Index(['recipe_id', 'serial', 'steps', 'id'], dtype='object'))

In [31]:
ingredients_df = ingredients_df.rename(columns={"ingredient_id": "recipe_id"})

In [53]:
recipe_step_to_meal = pd.merge(
    recipe_step_df,
    meal_to_recipe_df,
    on='recipe_id',
    how='left'
)

recipe_step_to_meal = recipe_step_to_meal.drop(columns=["recipe_id"])
recipe_step_to_meal = recipe_step_to_meal.drop_duplicates()


In [54]:
#  First merge: meals with recipes
meal_to_recipe = pd.merge(
    meals_df,
    meal_to_recipe_df,
    on='meal_id',  # Assuming this is the common field
    how='inner'   ,     # Keep all meals, even if no recipe match
)

# print(meal_to_recipe.head(5))
print(len(meal_to_recipe))

# # Second merge: recipes with ingredients
meal_to_recipe = pd.merge(
    meal_to_recipe,
    recipes_df,
    on='recipe_id',  # Assuming this is the common field
    how='inner'       # Keep all recipes, even if no ingredient match
)

len(meal_to_recipe)
# # Third merge: recipes with ingredients
final_df = pd.merge(
    meal_to_recipe,
    ingredients_df,
    on='recipe_id',  # Assuming this is the common field
    how='inner'       # Keep all ingredients, even if no ingredient match
)

final_df = pd.merge(
    final_df,
    recipe_step_to_meal,
    on='meal_id',
    how='left'
)

len(final_df)
# final_df = final_df.drop_duplicates()

112


3145

In [55]:
print(final_df.columns)

Index(['meal_id', 'name', 'description', 'status', 'created_at', 'updated_at',
       'source', 'video_url', 'recipe_id', 'course', 'cuisine', 'servings',
       'calories', 'prep time', 'cook time', 'total time', 'author', 'keyword',
       'soaking time', 'marinating time', 'cooling time', 'resting time',
       'wait time', 'main_ingredient', 'quantity', 'weight',
       'alternative_ingredient', 'measurement', 'serial', 'steps', 'id'],
      dtype='object')


In [57]:
final_df.to_csv("merged.csv", index=False)

In [77]:
# break down the datasets into meals table, recipes table and ingredients table
meals = final_df[["meal_id", "name", "description"]]
ingredients = final_df[["meal_id", "recipe_id", "main_ingredient", "alternative_ingredient"]]

In [79]:
import uuid

ingredients["ingredient_id"] = range(1, len(ingredients) + 1)
print(ingredients)


                                  meal_id  \
0    a6213618-a4b0-4b70-a132-a0cf4763bd02   
1    a6213618-a4b0-4b70-a132-a0cf4763bd02   
2    a6213618-a4b0-4b70-a132-a0cf4763bd02   
3    a6213618-a4b0-4b70-a132-a0cf4763bd02   
4    a6213618-a4b0-4b70-a132-a0cf4763bd02   
..                                    ...   
409  34f02b7c-7d18-46c4-abee-ff088272ec6b   
410  151e819a-7534-4598-b7f1-4f532670bb98   
411  7993b368-d196-4bb4-8766-dd1b94b7f175   
412  8fa0abd8-b6d6-4c72-9a11-d1b891b2f09b   
413  c258ef0d-0301-469a-bf2c-e85c2a19c641   

                                recipe_id     main_ingredient  \
0    2c2703b4-ed39-468d-a09b-4dda480e3bc7           plantains   
1    2c2703b4-ed39-468d-a09b-4dda480e3bc7                milk   
2    2c2703b4-ed39-468d-a09b-4dda480e3bc7                lime   
3    2c2703b4-ed39-468d-a09b-4dda480e3bc7  vanilla bean paste   
4    2c2703b4-ed39-468d-a09b-4dda480e3bc7   all purpose flour   
..                                    ...                 ...   
409 

/tmp/ipykernel_4558/2785697564.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingredients["ingredient_id"] = range(1, len(ingredients) + 1)


In [42]:
ingredients_cleaned = pd.read_json(DIRTY_INGREDIENTS)
ingredients_ai = pd.read_json(INGREDIENTS)

print(len(ingredients_cleaned), len(ingredients_ai))

2445 735


In [6]:
! pip install openpyxl
import pandas as pd


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [7]:
import os
import pandas as pd

# os.mkdir("../data/processed/final_csvs")
relationship_data = pd.ExcelFile("../data/processed/DigestDB.xlsx")

for sheet_name in relationship_data.sheet_names:
    data = pd.read_excel("../data/processed/DigestDB.xlsx", sheet_name=sheet_name)
    data.to_csv(f"../data/processed/final_csvs/{sheet_name}.csv", index=False)
    
    print(f"Saved {sheet_name}.csv")

Saved AlternativeIngredients.csv
Saved Category.csv
Saved CategoryToMeal.csv
Saved Cuisines.csv
Saved CuisineToMeal.csv
Saved Ingredient.csv
Saved Meal.csv
Saved RecipeStep.csv
Saved Measurements.csv
Saved Recipe.csv
Saved RecipeIngredient.csv


In [4]:
import pandas as pd

ingredients = pd.read_csv("../data/processed/final_csvs/Ingredient.csv")
ingredients = ingredients[["id", "name"]].drop_duplicates(subset=["name"])
ingredients.to_csv("../data/processed/final_csvs/Ingredient.csv", index=False)

In [5]:
from __future__ import annotations

import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="../../.env.dev")

DB_USER = os.environ["DB__USER"]
DB_PASSWORD = os.environ["DB__PASSWORD"]
DB_HOST = os.environ["DB__HOST"]
DB_PORT = os.environ["DB__PORT"]
DB_NAME = os.environ["DB__NAME"]

print(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_PASSWORD)

# engine = create_engine(url=f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# excel_file = "/media/cirec/CIREC/DigestDB.xlsx"
# sheets = pd.read_excel(excel_file, sheet_name=None)

# for sheet_name, df in sheets.items():
#     df.columns = [col.lower().replace(" ", "_") for col in df.columns]
#     df.to_sql(sheet_name, engine, if_exists="replace", index=False)
#     print(f"✅ Successfully inserted {sheet_name} into PostgreSQL.")

# print("🚀 All sheets have been pushed to PostgreSQL!")


postgres.ovbhljucgrjuruhqyxqb _Cletus20 aws-0-us-west-1.pooler.supabase.com 6543 _Cletus20


In [2]:
%reload_ext autoreload
%autoreload 2

from neo4j import GraphDatabase
from dotenv import load_dotenv
from neo4j import GraphDatabase
import os

load_dotenv(dotenv_path="../../.env.dev")

username = os.environ["NEO4J__USERNAME"]
password = os.environ["NEO4J__PASSWORD"]

DB_USER = os.environ["DB__USER"]
DB_PASSWORD = os.environ["DB__PASSWORD"]
DB_HOST = os.environ["DB__HOST"]
DB_PORT = os.environ["DB__PORT"]
DB_NAME = os.environ["DB__NAME"]

PATH = "../../"

queries_to = [
    '''
        WITH $jdbc_url AS url
        CALL apoc.load.jdbc(url, 'SELECT * FROM "Ingredient"') YIELD row
        CREATE (:Ingredient {ingredientId: row.ingredient_id, ingredientName: row.main_ingredient})
    ''',
    # '''CREATE INDEX ON :Ingredient(id)'''

    # '''
    #     WITH "jdbc:postgresql://''' + DB_USER + ''':''' + DB_PASSWORD + '''@''' + DB_HOST + ''':''' + DB_PORT + '''/''' + DB_NAME + '''" AS url
    #     CALL apoc.load.jdbc(url, 'SELECT * FROM "Meal"') YIELD row
    #     CREATE (:Meal {mealId: row.meal_id, mealName: row.name, description: row.description})
    # ''',
    # '''
    #     WITH "jdbc:postgresql://''' + DB_USER + ''':''' + DB_PASSWORD + '''@''' + DB_HOST + ''':''' + DB_PORT + '''/''' + DB_NAME + '''" AS url
    #     CALL apoc.load.jdbc(url, 'SELECT * FROM "Recipe"') YIELD row
    #     CREATE (:Recipe {recipeId: row.recipeid, author: row.author});
    # ''',
    # '''
    #     WITH "jdbc:postgresql://''' + DB_USER + ''':''' + DB_PASSWORD + '''@''' + DB_HOST + ''':''' + DB_PORT + '''/''' + DB_NAME + '''" AS url
    #     CALL apoc.load.jdbc(url, 'SELECT * FROM "Category"') YIELD row
    #     CREATE (:Category {categoryId: row.category_id, categoryName: row.category})
    # ''',
    # '''
    #     WITH "jdbc:postgresql://''' + DB_USER + ''':''' + DB_PASSWORD + '''@''' + DB_HOST + ''':''' + DB_PORT + '''/''' + DB_NAME + '''" AS url
    #     CALL apoc.load.jdbc(url, 'SELECT * FROM "Cuisines"') YIELD row
    #     CREATE (:Cuisines {cuisinesId: row.cuisines_id, cuisinesName: row.cuisines})
    # '''
]

# def index_graph_db_from_postgres():
#     with driver.session() as session:
#         # with open("../../queries_to_load_db.cypher", "r") as cypher_file:
#             # cypher_query = cypher_file.read()
#         for query in queries:
#             result = session.run(
#                 query=query, 
#                 USER=DB_USER, 
#                 PASSWORD=DB_PASSWORD, 
#                 PORT=DB_PORT,
#                 URL=DB_HOST,
#                 DATABASE=DB_NAME
#             )
#             for record in result:
#                 print(record)
            

# index_graph_db_from_postgres()

In [ ]:
# queries = [
    # '''
    #     WITH "jdbc:postgresql://''' + DB_USER + ''':''' + DB_PASSWORD + '''@''' + DB_HOST + ''':''' + DB_PORT + '''/''' + DB_NAME + '''" AS url
    #     CALL apoc.load.jdbc(url, 'SELECT * FROM "CategoryToMeal"') YIELD row
    #     MATCH (c:Category {categoryId: row.category_id})
    #     MATCH (m:Meal {mealId: row.meal_id})
    #     MERGE (m) - [:BELONGS_TO] -> (c);
    # ''',
    # '''
    #     WITH "jdbc:postgresql://''' + DB_USER + ''':''' + DB_PASSWORD + '''@''' + DB_HOST + ''':''' + DB_PORT + '''/''' + DB_NAME + '''" AS url
    #     CALL apoc.load.jdbc(url, 'SELECT * FROM "CuisineToMeal"') YIELD row
    #     MATCH (cu:Cuisines {cuisinesId: row.cuisines_id})
    #     MATCH (m:Meal {mealId: row.meal_id})
    #     MERGE (m) - [:HAS_CUISINE] -> (cu);    
    # ''',
    # '''
    #     WITH "jdbc:postgresql://''' + DB_USER + ''':''' + DB_PASSWORD + '''@''' + DB_HOST + ''':''' + DB_PORT + '''/''' + DB_NAME + '''" AS url
    #     CALL apoc.load.jdbc(url, 'SELECT * FROM "RecipeIngredient"') YIELD row
    #     MATCH (i:Ingredient {ingredientId: row.ingredient_id})
    #     MATCH (r:Recipe {recipeId: row.recipe_id})
    #     MERGE (r) - [:HAS_INGREDIENT] -> (i);
    # ''',
    # '''
    #     WITH "jdbc:postgresql://''' + DB_USER + ''':''' + DB_PASSWORD + '''@''' + DB_HOST + ''':''' + DB_PORT + '''/''' + DB_NAME + '''" AS url
    #     CALL apoc.load.jdbc(url, 'SELECT * FROM "Recipe"') YIELD row
    #     MATCH (r:Recipe {recipeId: row.recipeid})
    #     MATCH (m:Meal {mealId: row.meal_id})
    #     MERGE (m) - [:BELONGS_TO_RECIPE] -> (r);
    # '''
# ]

# def index_graph_db_from_postgres():
#     with driver.session() as session:
#         # with open("../../queries_to_load_db.cypher", "r") as cypher_file:
#             # cypher_query = cypher_file.read()
#         for query in queries:
#             result = session.run(
#                 query=query, 
#                 USER=DB_USER, 
#                 PASSWORD=DB_PASSWORD, 
#                 PORT=DB_PORT,
#                 URL=DB_HOST,
#                 DATABASE=DB_NAME
#             )
#             print("done!")
#             for record in result:
#                 print(record)
            

# index_graph_db_from_postgres()

done!


In [5]:
from neo4j import AsyncGraphDatabase
from typing import List
import sys
from neo4j.debug import watch

url = "bolt://localhost:7687"

JDBC_URL = f"jdbc:postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

watch("neo4j", out=sys.stdout)
AUTH = (os.environ["NEO4J__USERNAME"], os.environ["NEO4J__PASSWORD"])

async def create_data(q):
    async def create_node_and_relationships(tx, queries: List[str], params: dict):    
        for query in queries:
            await tx.run(query, **params)

    async with AsyncGraphDatabase.driver(uri=url, auth=AUTH) as driver:
        async with driver.session() as session:
            params ={
                "jdbc_url":JDBC_URL
            }
            await session.execute_write(create_node_and_relationships, q, params)

if __name__ == "__main__":
    await create_data(q=queries_to)

[DEBUG   ] [Thread 133809198993536] [Task 133808164483136] 2025-02-01 15:52:54,521  [#0000]  _: <POOL> created, direct address IPv4Address(('localhost', 7687))
[DEBUG   ] [Thread 133809198993536] [Task 133808164483136] 2025-02-01 15:52:54,521  [#0000]  _: <POOL> created, direct address IPv4Address(('localhost', 7687))
[DEBUG   ] [Thread 133809198993536] [Task 133808164483136] 2025-02-01 15:52:54,521  [#0000]  _: <POOL> created, direct address IPv4Address(('localhost', 7687))
[DEBUG   ] [Thread 133809198993536] [Task 133808164483136] 2025-02-01 15:52:54,526  [#0000]  _: <POOL> acquire direct connection, access_mode='WRITE', database=None
[DEBUG   ] [Thread 133809198993536] [Task 133808164483136] 2025-02-01 15:52:54,526  [#0000]  _: <POOL> acquire direct connection, access_mode='WRITE', database=None
[DEBUG   ] [Thread 133809198993536] [Task 133808164483136] 2025-02-01 15:52:54,526  [#0000]  _: <POOL> acquire direct connection, access_mode='WRITE', database=None
[DEBUG   ] [Thread 133809